In [1]:
import json
import re
from pathlib import Path
from typing import List, Optional

import requests
from bs4 import BeautifulSoup
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from paddleocr import PaddleOCRVL
from pydantic import BaseModel, Field
from tenacity import retry, stop_after_attempt, wait_fixed
from tqdm import tqdm

c:\Workdir\Projects\OCR_test\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Checking connectivity to the model hosters, this may take a while. To bypass this check, set `DISABLE_MODEL_SOURCE_CHECK` to `True`.


In [4]:
url = "http://localhost:8118/v1/models"

try:
    response = requests.get(url, timeout=5)

    print(f"Статус-код: {response.status_code}")
    print(f"Заголовки ответа:\n{response.headers}")

    if response.status_code == 200:
        print("✅ Эндпоинт доступен!")
        try:
            data = response.json()
            print("Данные ответа:")
            print(data)
        except ValueError:
            print("Ответ не в формате JSON")
            print(response.text)
    else:
        print(f"❌ Ошибка HTTP: {response.status_code}")
        if response.text.strip():
            print(f"Текст ошибки:\n{response.text}")
        else:
            print("Нет текста в ответе")
except:
    print("Эндпоинт не доступен")

Статус-код: 200
Заголовки ответа:
{'date': 'Mon, 05 Jan 2026 14:15:06 GMT', 'server': 'uvicorn', 'content-length': '475', 'content-type': 'application/json'}
✅ Эндпоинт доступен!
Данные ответа:
{'object': 'list', 'data': [{'id': 'PaddleOCR-VL-0.9B', 'object': 'model', 'created': 1767622507, 'owned_by': 'vllm', 'root': '/models', 'parent': None, 'max_model_len': 16384, 'permission': [{'id': 'modelperm-ca85e55e8b1b4285bfa6adcca2e52291', 'object': 'model_permission', 'created': 1767622507, 'allow_create_engine': False, 'allow_sampling': True, 'allow_logprobs': True, 'allow_search_indices': False, 'allow_view': True, 'allow_fine_tuning': False, 'organization': '*', 'group': None, 'is_blocking': False}]}]}


In [5]:
def simple_html_table_to_markdown(table) -> str:
    rows = []
    for tr in table.find_all("tr"):
        cells = tr.find_all(["td", "th"])
        row = [cell.get_text(strip=True).replace("\n", " ") for cell in cells]
        rows.append(row)

    if not rows:
        return ""

    max_cols = max(len(row) for row in rows) if rows else 0
    for row in rows:
        while len(row) < max_cols:
            row.append("")

    lines = []
    for i, row in enumerate(rows):
        line = "| " + " | ".join(row) + " |"
        lines.append(line)
        if i == 0:
            separator = "| " + " | ".join(["---"] * max_cols) + " |"
            lines.append(separator)
    return "\n".join(lines)


def html_to_markdown_with_tables(html: str) -> str:
    soup = BeautifulSoup(html, "html.parser")
    output = []

    for element in soup.children:
        if element.name == "table":
            md_table = simple_html_table_to_markdown(element)
            output.append(md_table)
            output.append("")
        else:
            text = element.get_text(strip=True)
            if text:
                output.append(text)
                output.append("")

    return "\n".join(output)

In [ ]:
input_files = [
    "./data/6.pdf",
]

output_path = Path("./results_paddler")
output_path.mkdir(parents=True, exist_ok=True)

pipeline = PaddleOCRVL(
    vl_rec_backend="vllm-server",
    vl_rec_server_url="http://localhost:8118/v1",
    vl_rec_model_name="PaddleOCR-VL-0.9B",
    layout_detection_model_name="PP-DocLayoutV2",
    layout_detection_model_dir="PP-DocLayoutV2",
    doc_orientation_classify_model_name="PP-LCNet_x1_0_doc_ori",
    doc_orientation_classify_model_dir="PP-LCNet_x1_0_doc_ori",
    use_doc_orientation_classify=True,
    use_doc_unwarping=False,
    use_layout_detection=True,
    layout_threshold=0.5,
    layout_nms=True,
    layout_unclip_ratio=[1.1, 1.0],
    layout_merge_bboxes_mode="union",
)

all_markdown_list = []

for input_file in tqdm(input_files):
    output = pipeline.predict(input=input_file)
    for res in output:
        md_info = res.markdown
        all_markdown_list.append(md_info)

final_markdown = pipeline.concatenate_markdown_pages(all_markdown_list)

base_name = "_".join(Path(f).stem for f in input_files)
mkd_file_path = output_path / f"{base_name}.md"

markdown_result = html_to_markdown_with_tables(final_markdown)

with open(mkd_file_path, "w", encoding="utf-8") as f:
    f.write(markdown_result)

Creating model: ('PP-LCNet_x1_0_doc_ori', 'PP-LCNet_x1_0_doc_ori')
Creating model: ('PP-DocLayoutV2', 'PP-DocLayoutV2')
Creating model: ('PaddleOCR-VL-0.9B', None)
100%|██████████| 1/1 [00:06<00:00,  6.73s/it]


In [ ]:
class BalanceHeadTable(BaseModel):
    organization: Optional[str] = Field(None, alias="Организация", description="Название организации")
    taxpayer_id: Optional[int] = Field(None, alias="Учетный номер плательщика", description="Учетный номер плательщика")
    economic_activity: Optional[str] = Field(
        None,
        alias="Вид экономической деятельности",
        description="Вид экономической деятельности",
    )
    legal_form: Optional[str] = Field(
        None,
        alias="Организационно-правовая форма",
        description="Организационно-правовая форма",
    )
    governing_body: Optional[str] = Field(None, alias="Орган управления", description="Орган управления")
    unit: Optional[str] = Field(None, alias="Единица измерения", description="Единица измерения")
    address: Optional[str] = Field(None, alias="Адрес", description="Адрес")


class BalanceDatesTable(BaseModel):
    approval_date: Optional[str] = Field(
        None,
        alias="Дата утверждения",
        description="Дата утверждения в формате ДД.ММ.ГГГГ",
    )
    submission_date: Optional[str] = Field(None, alias="Дата отправки", description="Дата отправки в формате ДД.ММ.ГГГГ")
    acceptance_date: Optional[str] = Field(None, alias="Дата принятия", description="Дата принятия в формате ДД.ММ.ГГГГ")


class BalanceMainTable(BaseModel):
    code_110: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="110",
        description="Основные средства",
        min_length=2,
        max_length=2,
    )
    code_120: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="120",
        description="Нематериальные активы",
        min_length=2,
        max_length=2,
    )
    code_130: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="130",
        description="Доходные вложения в материальные активы",
        min_length=2,
        max_length=2,
    )
    code_131: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="131",
        description="Инвестиционная недвижимость",
        min_length=2,
        max_length=2,
    )
    code_132: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="132",
        description="Предметы финансовой аренды (лизинга)",
        min_length=2,
        max_length=2,
    )
    code_133: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="133",
        description="Прочие доходные вложения в материальные активы",
        min_length=2,
        max_length=2,
    )
    code_140: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="140",
        description="Вложения в долгосрочные активы",
        min_length=2,
        max_length=2,
    )
    code_150: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="150",
        description="Долгосрочные финансовые вложения",
        min_length=2,
        max_length=2,
    )
    code_160: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="160",
        description="Отложенные налоговые активы",
        min_length=2,
        max_length=2,
    )
    code_170: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="170",
        description="Долгосрочная дебиторская задолженность",
        min_length=2,
        max_length=2,
    )
    code_180: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="180",
        description="Прочие долгосрочные активы",
        min_length=2,
        max_length=2,
    )
    code_190: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="190",
        description="Итого по разделу I",
        min_length=2,
        max_length=2,
    )
    code_210: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="210",
        description="Запасы",
        min_length=2,
        max_length=2,
    )
    code_211: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="211",
        description="Материалы",
        min_length=2,
        max_length=2,
    )
    code_212: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="212",
        description="Животные на выращивании и откорме",
        min_length=2,
        max_length=2,
    )
    code_213: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="213",
        description="Незавершенное производство",
        min_length=2,
        max_length=2,
    )
    code_214: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="214",
        description="Готовая продукция и товары",
        min_length=2,
        max_length=2,
    )
    code_215: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="215",
        description="Товары отгруженные",
        min_length=2,
        max_length=2,
    )
    code_216: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="216",
        description="Прочие запасы",
        min_length=2,
        max_length=2,
    )
    code_220: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="220",
        description="Долгосрочные активы, предназначенные для реализации",
        min_length=2,
        max_length=2,
    )
    code_230: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="230",
        description="Расходы будущих периодов",
        min_length=2,
        max_length=2,
    )
    code_240: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="240",
        description="Налог на добавленную стоимость по приобретенным товарам, работам, услугам",
        min_length=2,
        max_length=2,
    )
    code_250: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="250",
        description="Краткосрочная дебиторская задолженность",
        min_length=2,
        max_length=2,
    )
    code_260: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="260",
        description="Краткосрочные финансовые вложения",
        min_length=2,
        max_length=2,
    )
    code_270: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="270",
        description="Денежные средства и эквиваленты денежных средств",
        min_length=2,
        max_length=2,
    )
    code_280: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="280",
        description="Прочие краткосрочные активы",
        min_length=2,
        max_length=2,
    )
    code_290: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="290",
        description="Итого по разделу II",
        min_length=2,
        max_length=2,
    )
    code_300: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="300",
        description="Баланс",
        min_length=2,
        max_length=2,
    )
    code_410: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="410",
        description="Уставный капитал",
        min_length=2,
        max_length=2,
    )
    code_420: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="420",
        description="Неоплаченная часть уставного капитала",
        min_length=2,
        max_length=2,
    )
    code_430: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="430",
        description="Собственные акции (доли в уставном капитале)",
        min_length=2,
        max_length=2,
    )
    code_440: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="440",
        description="Резервный капитал",
        min_length=2,
        max_length=2,
    )
    code_450: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="450",
        description="Добавочный капитал",
        min_length=2,
        max_length=2,
    )
    code_460: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="460",
        description="Нераспределенная прибыль (непокрытый убыток)",
        min_length=2,
        max_length=2,
    )
    code_470: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="470",
        description="Чистая прибыль (убыток) отчетного периода",
        min_length=2,
        max_length=2,
    )
    code_480: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="480",
        description="Целевое финансирование",
        min_length=2,
        max_length=2,
    )
    code_490: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="490",
        description="Итого по разделу III",
        min_length=2,
        max_length=2,
    )
    code_510: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="510",
        description="Долгосрочные кредиты и займы",
        min_length=2,
        max_length=2,
    )
    code_520: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="520",
        description="Долгосрочные обязательства по лизинговым платежам",
        min_length=2,
        max_length=2,
    )
    code_530: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="530",
        description="Отложенные налоговые обязательства",
        min_length=2,
        max_length=2,
    )
    code_540: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="540",
        description="Доходы будущих периодов",
        min_length=2,
        max_length=2,
    )
    code_550: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="550",
        description="Резервы предстоящих платежей",
        min_length=2,
        max_length=2,
    )
    code_560: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="560",
        description="Прочие долгосрочные обязательства",
        min_length=2,
        max_length=2,
    )
    code_590: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="590",
        description="Итого по разделу IV",
        min_length=2,
        max_length=2,
    )
    code_610: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="610",
        description="Краткосрочные кредиты и займы",
        min_length=2,
        max_length=2,
    )
    code_620: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="620",
        description="Краткосрочная часть долгосрочных обязательств",
        min_length=2,
        max_length=2,
    )
    code_630: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="630",
        description="Краткосрочная кредиторская задолженность",
        min_length=2,
        max_length=2,
    )
    code_631: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="631",
        description="Поставщикам, подрядчикам, исполнителям",
        min_length=2,
        max_length=2,
    )
    code_632: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="632",
        description="По авансам полученным",
        min_length=2,
        max_length=2,
    )
    code_633: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="633",
        description="По налогам и сборам",
        min_length=2,
        max_length=2,
    )
    code_634: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="634",
        description="По социальному страхованию и обеспечению",
        min_length=2,
        max_length=2,
    )
    code_635: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="635",
        description="По оплате труда",
        min_length=2,
        max_length=2,
    )
    code_636: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="636",
        description="По лизинговым платежам",
        min_length=2,
        max_length=2,
    )
    code_637: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="637",
        description="Собственнику имущества (учредителям, участникам)",
        min_length=2,
        max_length=2,
    )
    code_638: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="638",
        description="Прочим кредиторам",
        min_length=2,
        max_length=2,
    )
    code_640: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="640",
        description="Обязательства, предназначенные для реализации",
        min_length=2,
        max_length=2,
    )
    code_650: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="650",
        description="Доходы будущих периодов",
        min_length=2,
        max_length=2,
    )
    code_660: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="660",
        description="Резервы предстоящих платежей",
        min_length=2,
        max_length=2,
    )
    code_670: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="670",
        description="Прочие краткосрочные обязательства",
        min_length=2,
        max_length=2,
    )
    code_690: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="690",
        description="Итого по разделу V",
        min_length=2,
        max_length=2,
    )
    code_700: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="700",
        description="Баланс",
        min_length=2,
        max_length=2,
    )


class ReportMainTable(BaseModel):
    code_010: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="010",
        description="Выручка от реализации продукции, товаров, работ, услуг",
        min_length=2,
        max_length=2,
    )
    code_020: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="020",
        description="Себестоимость реализованной продукции, товаров, работ, услуг",
        min_length=2,
        max_length=2,
    )
    code_030: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="030",
        description="Валовая прибыль",
        min_length=2,
        max_length=2,
    )
    code_040: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="040",
        description="Управленческие расходы",
        min_length=2,
        max_length=2,
    )
    code_050: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="050",
        description="Расходы на реализацию",
        min_length=2,
        max_length=2,
    )
    code_060: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="060",
        description="Прибыль (убыток) от реализации продукции, товаров, работ, услуг",
        min_length=2,
        max_length=2,
    )
    code_070: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="070",
        description="Прочие доходы по текущей деятельности",
        min_length=2,
        max_length=2,
    )
    code_080: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="080",
        description="Прочие расходы по текущей деятельности",
        min_length=2,
        max_length=2,
    )
    code_090: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="090",
        description="Прибыль (убыток) от текущей деятельности",
        min_length=2,
        max_length=2,
    )
    code_100: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="100",
        description="Доходы по инвестиционной деятельности",
        min_length=2,
        max_length=2,
    )
    code_101: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="101",
        description="Доходы от выбытия основных средств, нематериальных активов и других долгосрочных активов",
        min_length=2,
        max_length=2,
    )
    code_102: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="102",
        description="Доходы от участия в уставном капитале других организаций",
        min_length=2,
        max_length=2,
    )
    code_103: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="103",
        description="Проценты к получению",
        min_length=2,
        max_length=2,
    )
    code_104: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="104",
        description="Прочие доходы по инвестиционной деятельности",
        min_length=2,
        max_length=2,
    )
    code_110: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="110",
        description="Расходы по инвестиционной деятельности",
        min_length=2,
        max_length=2,
    )
    code_111: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="111",
        description="Расходы от выбытия основных средств, нематериальных активов и других долгосрочных активов",
        min_length=2,
        max_length=2,
    )
    code_112: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="112",
        description="Прочие расходы по инвестиционной деятельности",
        min_length=2,
        max_length=2,
    )
    code_120: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="120",
        description="Доходы по финансовой деятельности",
        min_length=2,
        max_length=2,
    )
    code_121: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="121",
        description="Курсовые разницы от пересчета активов и обязательств",
        min_length=2,
        max_length=2,
    )
    code_122: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="122",
        description="Прочие доходы по финансовой деятельности",
        min_length=2,
        max_length=2,
    )
    code_130: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="130",
        description="Расходы по финансовой деятельности",
        min_length=2,
        max_length=2,
    )
    code_131: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="131",
        description="Проценты к уплате",
        min_length=2,
        max_length=2,
    )
    code_132: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="132",
        description="Курсовые разницы от пересчета активов и обязательств",
        min_length=2,
        max_length=2,
    )
    code_133: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="133",
        description="Прочие расходы по финансовой деятельности",
        min_length=2,
        max_length=2,
    )
    code_140: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="140",
        description="Прибыль (убыток) от инвестиционной и финансовой деятельности",
        min_length=2,
        max_length=2,
    )
    code_150: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="150",
        description="Прибыль (убыток) до налогообложения",
        min_length=2,
        max_length=2,
    )
    code_160: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="160",
        description="Налог на прибыль",
        min_length=2,
        max_length=2,
    )
    code_170: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="170",
        description="Изменение отложенных налоговых активов",
        min_length=2,
        max_length=2,
    )
    code_180: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="180",
        description="Изменение отложенных налоговых обязательств",
        min_length=2,
        max_length=2,
    )
    code_190: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="190",
        description="Прочие налоги и сборы, исчисляемые из прибыли (дохода)",
        min_length=2,
        max_length=2,
    )
    code_200: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="200",
        description="Прочие платежи, исчисляемые из прибыли (дохода)",
        min_length=2,
        max_length=2,
    )
    code_210: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="210",
        description="Чистая прибыль (убыток)",
        min_length=2,
        max_length=2,
    )
    code_220: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="220",
        description="Результат от переоценки долгосрочных активов, не включаемый в чистую прибыль (убыток)",
        min_length=2,
        max_length=2,
    )
    code_230: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="230",
        description="Результат от прочих операций, не включаемый в чистую прибыль (убыток)",
        min_length=2,
        max_length=2,
    )
    code_240: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="240",
        description="Совокупная прибыль (убыток)",
        min_length=2,
        max_length=2,
    )
    code_250: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="250",
        description="Базовая прибыль (убыток) на акцию",
        min_length=2,
        max_length=2,
    )
    code_260: List[Optional[int]] = Field(
        default_factory=lambda: [None, None],
        alias="260",
        description="Разводненная прибыль (убыток) на акцию",
        min_length=2,
        max_length=2,
    )


class TablesData(BaseModel):
    balance_head_table: BalanceHeadTable
    balance_dates_table: BalanceDatesTable
    balance_main_table_dates: List[Optional[str]] = Field(
        ...,
        description="Даты, соответствующие двум столбцам основной таблицы баланса в формате ДД.ММ.ГГГГ",
    )
    balance_main_table: BalanceMainTable
    report_main_table: ReportMainTable


class ParsedPDF(BaseModel):
    tables_data: TablesData

In [63]:
output_parser = PydanticOutputParser(pydantic_object=ParsedPDF)
format_instructions = output_parser.get_format_instructions()

In [ ]:
system_prompt = """
Ты — эксперт в извлечении структурированных финансовых данных из форм бухгалтерской отчётности в Markdown формате.
Твоя задача — преобразовать текстовое (табличное) описание в строго валидный JSON формата ParsedPDF:

1. **Найди Бухгалтерский баланс и извлеки 4 компонента**:
   - `balance_head_table` — шапка баланса (если не указаны — ключ остается, значение - null):
        - организация, 
        - учетный номер плательщика, 
        - вид экономической деятельности, 
        - организационно-правовая форма, 
        - орган управления, 
        - единица измерения, 
        - адрес.
   - `balance_dates_table` (если не указаны — ключ остается, значение - null):
        - дата утверждения, 
        - дата отправки, 
        - дата принятия.
   - `balance_main_table_dates` — **две даты из заголовков колонок баланса**:  
     *первая* — более поздняя (например, "30.06.2025"),  
     *вторая* — более ранняя (например, "31.12.2024").  
     Всегда выводи в порядке: **[более поздняя дата, более ранняя дата]** как строки в формате "ДД.ММ.ГГГГ".
   - `balance_main_table` — таблица из документа Бухгалтерский баланс: ключ — строковый код строки (например, "110", "470"), значение — **массив из двух элементов**:  
     **[значение за более позднюю дату, значение за более раннюю дату]**. Если значения неизвестны или нечитаемы, выводи массив **[null, null]**.
3. **Найди Отчёт о прибылях и убытках и извлеки 1 компонент:**
   - `report_main_table` — таблица из документа Отчёт о прибылях и убытках: : ключ — строковый код строки (например, "010", "120"), значение — **массив из двух элементов**:
     **[значение за более позднюю дату, значение за более раннюю дату]**. Если значения неизвестны или нечитаемы, выводи массив **[null, null]**.

4. **Правила обработки значений**:
   - Любые пропуски, любые прочерки, тире ("—", "-", "—", "–") заменяй на `null`.
   - Если значение в ячейке отсутствует, используй null, а не "-", "—", пустую строку или любой другой символ.
   - Пробелы в числах (например, "9 044") удаляй, преобразуй в целое число `9044`.
   - Отрицательным считается ТОЛЬКО число, начинающееся с символа минуса (`-`), например: `-186`.
   - Все коды строк — **всегда строки**, даже если состоят из цифр (например, "110", а не 110).
   - Если значение отсутствует или нечитаемо — ставь `null`.
   - Не интерпретируй, не агрегируй, не исправляй логические ошибки — только извлекай то, что видишь.

5. **Формат выхода**:
   - Верни **только валидный JSON**, строго соответствующий Pydantic-схеме.
   - Без пояснений, без комментариев, без markdown.
   - Убедись, что:
     ✓ Все поля присутствуют (даже если null),
     ✓ Нет лишних ключей,
     ✓ JSON валиден (проверь кавычки, запятые, скобки),
     ✓ В JSON отсутствуют тире ("—", "-", "—", "–"),
     ✓ Поля `balance_main_table` и `report_main_table` содержат **все коды строк**, присутствующие в документе (включая те, где оба значения null).

ВАЖНО:
- Если в исходном документе значение отсутствует, нечитаемо или указан прочерк — выводи `null`, не используй "-1", "0".
- НИКОГДА не придумывай числовые значения.
- Не используй примеры из памяти — только то, что видишь в документе.


{format_instructions}
"""

user_prompt = """
Проанализируй следующий отчет и извлеки структурированные данные строго по правилам:

{report}
"""

prompt = ChatPromptTemplate.from_messages([("system", system_prompt), ("user", user_prompt)])

In [65]:
def remove_parentheses_around_numbers(text):
    if not isinstance(text, str):
        return text

    def replace_match(m):
        inner = m.group(1)
        if re.fullmatch(r"[\d\s]+", inner.strip()):
            return inner
        else:
            return m.group(0)

    return re.sub(r"\(([^)]+)\)", replace_match, text)


def truncate_after_diluted_eps(markdown: str) -> str:
    lines = markdown.splitlines(keepends=True)
    pattern = "| Разводненная прибыль (убыток) на акцию | 260 |"

    for i, line in enumerate(lines):
        if line.strip().startswith(pattern):
            return "".join(lines[: i + 1])

    return markdown

In [66]:
def enrich_json(input_json_str):
    if isinstance(input_json_str, str):
        data = json.loads(input_json_str)
    else:
        data = input_json_str.copy()

    required_keys = [
        "balance_head_table",
        "balance_dates_table",
        "balance_main_table_dates",
        "balance_main_table",
        "report_main_table",
    ]

    tables_data = data.get("tables_data", {})
    message = {key: "OK" if key in tables_data else "Missing" for key in required_keys}

    new_data = {"message": message, "xlsx": None, **data}

    return new_data

In [67]:
with open("results_paddler/6.md", "r", encoding="utf-8") as file:
    full_table = file.read()

In [68]:
llm = ChatOpenAI(
    base_url="http://localhost:11434/v1",
    api_key="token-abc",
    model="gpt-oss:20b-new",
    temperature=0.0,
    top_p=0.8,
    max_tokens=4096,
    reasoning_effort="low",
    timeout=60,
)

In [ ]:
def fix_json_with_llm(format_instructions: str, broken_json_text: str) -> str:
    system_fix_prompt = """ 
    Ты AI помощник, который исправляет JSON.
    Верни **только валидный JSON**, строго соответствующий Pydantic-схеме.
    {format_instructions}
    """
    user_fix_prompt = """ 
    Исправь этот текст так, чтобы результат был корректным JSON:
    {broken_json_text}
    """
    prompt = ChatPromptTemplate.from_messages([("system", system_fix_prompt), ("user", user_fix_prompt)])

    messages = prompt.format_messages(format_instructions=format_instructions, broken_json_text=broken_json_text)
    fixed = llm.invoke(messages)
    return fixed.content


@retry(stop=stop_after_attempt(3), wait=wait_fixed(1))
def call_llm_and_parse(messages):
    result = llm.invoke(messages)
    raw_text = result.content

    try:
        return output_parser.parse(raw_text)
    except Exception as e:
        print("Primary parse failed. Attempting automatic JSON repair...")
        print(f"Original error: {e}")

        fixed_json = fix_json_with_llm(format_instructions, raw_text)

        try:
            return output_parser.parse(fixed_json)
        except Exception as e2:
            print(f"JSON repair also failed: {e2}")
            print(f"Repaired JSON candidate:\n{fixed_json}")
            raise

In [70]:
md_file = truncate_after_diluted_eps(remove_parentheses_around_numbers(full_table))

messages = prompt.format_messages(
    format_instructions=format_instructions,
    report=md_file,
)
try:
    parsed = call_llm_and_parse(messages)
    parsed = parsed.model_dump(by_alias=True)
except Exception:
    print("Failed even after repair and retries")


# result = llm.invoke(messages)
# raw_text = result.content
# output_parser.parse(raw_text)

Primary parse failed. Attempting automatic JSON repair...
Original error: Failed to parse ParsedPDF from completion {}. Got: 1 validation error for ParsedPDF
tables_data
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
For troubleshooting, visit: https://docs.langchain.com/oss/python/langchain/errors/OUTPUT_PARSING_FAILURE 


In [71]:
enriched_result_json = enrich_json(parsed)

In [72]:
results_dir = Path("results_paddler")
results_dir.mkdir(exist_ok=True)

if isinstance(input_files, list):
    base_name = "_".join(Path(p).stem for p in input_files)
else:
    base_name = Path(input_files).stem

output_path = results_dir / f"{base_name}.json"

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(enriched_result_json, f, ensure_ascii=False, indent=2)

Новый:
1: код 300
2_3: код 631
4_5: код 160, 180, 250
6: -

Вендор:
1: 12 ошибок
2_3: -
4_5: 4 ошибки
6: -